# Generate Color Bins For Each Layer

### 0. Import libraries and create empty dictionaries

In [11]:
import geopandas as gpd
import numpy as np
import mapclassify as mc
import matplotlib.pyplot as plt

In [12]:
total_dict = {}
county_dict = {}
state_dict = {}
dph_illinois_dict = {}
who_world_dict = {}
vul_dict = {}

In [13]:
dynamic_dict = {}
dynamic_county_dict = {}
dynamic_state_dict = {}
dynamic_illinois_dict = {}

### 1. For US County-level Data

#### 1.0 Load Data

In [14]:
counties_gdf = gpd.read_file('nyt_counties_data.geojson')

In [15]:
counties_gdf.head(5)

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,start,end,dt_unit,change_ts,geometry
0,Bladen,North Carolina,33778,37017,2020-04-03,2020-05-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",596.0,5.0,6.0,-1.0,2020-01-21,2020-08-05,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ..."
1,Stanly,North Carolina,61114,37167,2020-03-20,2020-04-09,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",938.0,10.0,24.0,13.0,2020-01-21,2020-08-05,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ..."
2,Summit,Ohio,541810,39153,2020-03-14,2020-03-27,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",3399.0,61.0,218.0,1.0,2020-01-21,2020-08-05,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ..."
3,Sullivan,Pennsylvania,6177,42113,2020-04-04,None,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",10.0,0.0,0.0,0.0,2020-01-21,2020-08-05,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ..."
4,Upshur,Texas,40769,48459,2020-03-20,2020-07-25,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",200.0,6.0,2.0,0.0,2020-01-21,2020-08-05,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ..."


In [16]:
case = counties_gdf['today_case']
death = counties_gdf['today_death']
case_per_100k_capita = counties_gdf['today_case']/(counties_gdf['population']/100000)
death_per_100k_capita = counties_gdf['today_death']/(counties_gdf['population']/100000)
# death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
# death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [17]:
log_case = case.apply(lambda x: np.log(x + 1))
log_death = death.apply(lambda x: np.log(x + 1))
log_case_per_100k_capita = case_per_100k_capita.apply(lambda x: np.log(x + 1))
log_death_per_100k_capita = death.apply(lambda x: np.log(x + 1))

#### 1.1 Get Constant Classes from today's data

In [18]:
def create_dict(column):
    tmp_dict = {}
    tmp_dict['Quantiles'] = {
        'bins': ','.join(mc.Quantiles(column, k=6).bins.astype(str)),
        'k': 6
    }
    tmp_dict['FisherJenks'] = {
        'bins': ','.join(mc.FisherJenks(column, k=6).bins.astype(str)),
        'k': 6
    }
    tmp_dict['NaturalBreaks'] = {
        'bins': ','.join(mc.NaturalBreaks(column, k=6).bins.astype(str)),
        'k': 6
    }
    return tmp_dict

In [19]:
county_dict['case'] = {}
county_dict['death'] = {}
county_dict['case_per_100k_capita'] = {}
county_dict['death_per_100k_capita'] = {}

In [20]:
county_dict['case']['nolog'] = create_dict(case)
county_dict['case']['log'] = create_dict(log_case)
county_dict['death']['nolog'] = create_dict(death)
county_dict['death']['log'] = create_dict(log_death)
county_dict['case_per_100k_capita']['nolog'] = create_dict(case_per_100k_capita)
county_dict['case_per_100k_capita']['log'] = create_dict(log_case_per_100k_capita)
county_dict['death_per_100k_capita']['nolog'] = create_dict(death_per_100k_capita)
county_dict['death_per_100k_capita']['log'] = create_dict(log_death_per_100k_capita)


In [21]:
county_dict

{'case': {'nolog': {'Quantiles': {'bins': '26.0,81.0,189.0,425.00000000000045,1240.0000000000005,231268.0',
    'k': 6},
   'FisherJenks': {'bins': '3033.0,10889.0,26109.0,59354.0,125948.0,231268.0',
    'k': 6},
   'NaturalBreaks': {'bins': '2958.0,10700.0,26109.0,59354.0,125948.0,231268.0',
    'k': 6}},
  'log': {'Quantiles': {'bins': '3.295836866004329,4.406719247264253,5.247024072160486,6.054439346269372,7.123672785204608,12.351336814161709',
    'k': 6},
   'FisherJenks': {'bins': '1.6094379124341003,3.6109179126442243,5.017279836814924,6.331501849893691,7.983440063006542,12.351336814161709',
    'k': 6},
   'NaturalBreaks': {'bins': '1.6094379124341003,3.58351893845611,4.955827057601261,6.267200548541362,7.929846429742503,12.351336814161709',
    'k': 6}}},
 'death': {'nolog': {'Quantiles': {'bins': '0.0,1.0,3.0,8.0,30.0,23034.0',
    'k': 6},
   'FisherJenks': {'bins': '117.0,487.0,1239.0,2811.0,4909.0,23034.0', 'k': 6},
   'NaturalBreaks': {'bins': '122.0,487.0,1239.0,2811.0,4

In [22]:
import json
total_dict['county'] = county_dict
total_dict['state'] = state_dict
total_dict['dph_illinois'] = dph_illinois_dict
total_dict['who_world'] = who_world_dict
total_dict['vulnerability'] = vul_dict
# dynamic_dict['county'] = dynamic_county_dict
# dynamic_dict['state'] = dynamic_state_dict
# dynamic_dict['illinois'] = dynamic_illinois_dict
with open('classes.json','w') as json_file:
    json.dump(total_dict, json_file)
# with open('dynamic_classes.json','w') as json_file:
#     json.dump(dynamic_dict, json_file)
print('done')
exit()

done


### 2. For US State-level Data

#### 2.0 Load Data

In [23]:
state_gdf = gpd.read_file('nyt_states_data.geojson')

In [24]:
state_gdf.head(5)

,index,NAME,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,start,end,dt_unit,change_ts,geometry
0,0,West Virginia,1829054,54,2020-03-17,2020-03-29,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",47842.0,845.0,735.0,6.0,2020-01-21,2020-11-30,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","POLYGON ((-80.51899 40.63880, -80.51934 39.721..."
1,1,Florida,20598139,12,2020-03-01,2020-03-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",999311.0,6659.0,18596.0,97.0,2020-01-21,2020-11-30,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","MULTIPOLYGON (((-82.98748 24.62538, -82.89993 ..."
2,2,Illinois,12821497,17,2020-01-24,2020-03-17,"0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",727825.0,6338.0,13015.0,120.0,2020-01-21,2020-11-30,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,-1.0,-1....","POLYGON ((-90.64284 42.50848, -87.01994 42.493..."
3,3,Minnesota,5527358,27,2020-03-06,2020-03-21,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",318822.0,5794.0,3652.0,15.0,2020-01-21,2020-11-30,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","POLYGON ((-97.22872 49.00056, -95.15371 48.998..."
4,4,Maryland,6003435,24,2020-03-05,2020-03-18,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",198710.0,1983.0,4641.0,16.0,2020-01-21,2020-11-30,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","POLYGON ((-77.71952 39.32131, -77.76086 39.340..."


#### 2.1 Get Constant Classes from today's data

In [25]:
state_case = state_gdf['today_case']
state_death = state_gdf['today_death']
state_case_per_100k_capita = state_gdf['today_case']/(state_gdf['population']/100000)
state_death_per_100k_capita = state_gdf['today_death']/(state_gdf['population']/100000)
#death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
#death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [26]:
state_death_per_100k_capita = state_death_per_100k_capita.replace(np.nan,0)
state_death_per_100k_capita = state_death_per_100k_capita.replace(np.inf,0)
state_case_per_100k_capita = state_case_per_100k_capita.replace(np.nan,0)
state_case_per_100k_capita = state_case_per_100k_capita.replace(np.inf,0)

In [27]:
state_log_case = state_case.apply(lambda x: np.log(x + 1))
state_log_death = state_death.apply(lambda x: np.log(x + 1))
state_log_case_per_100k_capita = state_case_per_100k_capita.apply(lambda x: np.log(x + 1))
state_log_death_per_100k_capita =state_death_per_100k_capita.apply(lambda x: np.log(x + 1))

In [28]:
state_dict['case'] = {}
state_dict['death'] = {}
state_dict['case_per_100k_capita'] = {}
state_dict['death_per_100k_capita'] = {}

In [29]:
state_dict['case']['nolog'] = create_dict(state_case)
state_dict['case']['log'] = create_dict(state_log_case)
state_dict['death']['nolog'] = create_dict(state_death)
state_dict['death']['log'] = create_dict(state_log_death)
state_dict['case_per_100k_capita']['nolog'] = create_dict(state_case_per_100k_capita)
state_dict['case_per_100k_capita']['log'] = create_dict(state_log_case_per_100k_capita)
state_dict['death_per_100k_capita']['nolog'] = create_dict(state_death_per_100k_capita)
state_dict['death_per_100k_capita']['log'] = create_dict(state_log_death_per_100k_capita)

In [30]:
state_dict

{'case': {'nolog': {'Quantiles': {'bins': '33305.00000000001,97095.00000000001,183255.0,237835.0000000001,366971.0000000001,1262145.0',
    'k': 6},
   'FisherJenks': {'bins': '117295.0,249524.0,454328.0,727825.0,999311.0,1262145.0',
    'k': 6},
   'NaturalBreaks': {'bins': '117295.0,249524.0,454328.0,727825.0,999311.0,1262145.0',
    'k': 6}},
  'log': {'Quantiles': {'bins': '10.413492839918217,11.483455458786198,12.11863936136841,12.379336639501627,12.813040829848221,14.048324004773729',
    'k': 6},
   'FisherJenks': {'bins': '4.672828834461906,8.96469555531546,10.775680095688484,11.98560675607755,13.026576884180775,14.048324004773729',
    'k': 6},
   'NaturalBreaks': {'bins': '4.672828834461906,8.96469555531546,10.481644668462405,11.762967973593053,13.026576884180775,14.048324004773729',
    'k': 6}}},
 'death': {'nolog': {'Quantiles': {'bins': '680.0,1039.0000000000002,2502.0,4381.000000000002,9198.000000000002,34150.0',
    'k': 6},
   'FisherJenks': {'bins': '1743.0,4062.0,664

### 3. For IDPH County-level Data

#### 3.0 Load Data

In [31]:
dph_illinois_gdf = gpd.read_file('illinois/dph_county_data.geojson')

In [ ]:
dph_illinois_gdf.head(5)

In [ ]:
dph_illinois_gdf = dph_illinois_gdf[dph_illinois_gdf['id'] != 'Illinois']

#### 3.1 Get Constant Classes from today's data

In [ ]:
illinois_case = dph_illinois_gdf['today_case']
illinois_death = dph_illinois_gdf['today_death']
illinois_case_per_100k_capita = dph_illinois_gdf['today_case']/(dph_illinois_gdf['population']/100000)
illinois_death_per_100k_capita = dph_illinois_gdf['today_death']/(dph_illinois_gdf['population']/100000)
#death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
#death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [ ]:
illinois_death_per_100k_capita = illinois_death_per_100k_capita.replace(np.nan,0)
illinois_death_per_100k_capita = illinois_death_per_100k_capita.replace(np.inf,0)
illinois_case_per_100k_capita = illinois_case_per_100k_capita.replace(np.nan,0)
illinois_case_per_100k_capita = illinois_case_per_100k_capita.replace(np.inf,0)

In [ ]:
illinois_log_case = illinois_case.apply(lambda x: np.log(x + 1))
illinois_log_death = illinois_death.apply(lambda x: np.log(x + 1))
illinois_log_case_per_100k_capita = illinois_case_per_100k_capita.apply(lambda x: np.log(x + 1))
illinois_log_death_per_100k_capita =illinois_death_per_100k_capita.apply(lambda x: np.log(x + 1))

In [ ]:
dph_illinois_dict['case'] = {}
dph_illinois_dict['death'] = {}
dph_illinois_dict['case_per_100k_capita'] = {}
dph_illinois_dict['death_per_100k_capita'] = {}

In [ ]:
dph_illinois_dict['case']['nolog'] = create_dict(illinois_case)
dph_illinois_dict['case']['log'] = create_dict(illinois_log_case)
dph_illinois_dict['death']['nolog'] = create_dict(illinois_death)
dph_illinois_dict['death']['log'] = create_dict(illinois_log_death)
dph_illinois_dict['case_per_100k_capita']['nolog'] = create_dict(illinois_case_per_100k_capita)
dph_illinois_dict['case_per_100k_capita']['log'] = create_dict(illinois_log_case_per_100k_capita)
dph_illinois_dict['death_per_100k_capita']['nolog'] = create_dict(illinois_death_per_100k_capita)
dph_illinois_dict['death_per_100k_capita']['log'] = create_dict(illinois_log_death_per_100k_capita)

In [ ]:
dph_illinois_dict

### 4. For IDPH Zipcode-level Case (Static)

#### 4.0 Load Data

In [ ]:
dph_zipcode_gdf = gpd.read_file('illinois/dph_zipcode_data.geojson')

In [ ]:
dph_zipcode_gdf

#### 4.1 Get Constant Classes from today's data

In [ ]:
dph_zipcode_tested = dph_zipcode_gdf['total_tested']
dph_zipcode_case = dph_zipcode_gdf['confirmed_cases']

In [ ]:
dph_zipcode_log_tested = dph_zipcode_tested.apply(lambda x: np.log(x + 1))
dph_zipcode_log_case = dph_zipcode_case.apply(lambda x: np.log(x + 1))

In [ ]:
dph_illinois_dict['tested'] = {}
dph_illinois_dict['zipcode_case'] = {}
dph_illinois_dict['zipcode_tested'] = {}

In [ ]:
dph_illinois_dict['zipcode_case']['nolog'] = create_dict(dph_zipcode_case)
dph_illinois_dict['zipcode_case']['log'] = create_dict(dph_zipcode_log_case)
dph_illinois_dict['zipcode_tested']['nolog'] = create_dict(dph_zipcode_tested)
dph_illinois_dict['zipcode_tested']['log'] = create_dict(dph_zipcode_log_tested)

In [ ]:
dph_illinois_dict

### 5. For WHO World Data

#### 5.0 Load Data

In [ ]:
who_world_gdf = gpd.read_file('worldwide/who_world_data.geojson')

#### 5.1 Get Constant Classes from today's data

In [ ]:
who_world_case = who_world_gdf['today_case']
who_world_death = who_world_gdf['today_death']
who_world_case_per_100k_capita = who_world_gdf['today_case']/(who_world_gdf['population']/100000)
who_world_death_per_100k_capita = who_world_gdf['today_death']/(who_world_gdf['population']/100000)

In [ ]:
who_world_log_case = who_world_case.apply(lambda x: np.log(x + 1))
who_world_log_death = who_world_death.apply(lambda x: np.log(x + 1))
who_world_log_case_per_100k_capita = who_world_case_per_100k_capita.apply(lambda x: np.log(x + 1))
who_world_log_death_per_100k_capita = who_world_death_per_100k_capita.apply(lambda x: np.log(x + 1))

In [ ]:
who_world_dict['case'] = {}
who_world_dict['death'] = {}
who_world_dict['case_per_100k_capita'] = {}
who_world_dict['death_per_100k_capita'] = {}

In [ ]:
who_world_dict['case']['nolog'] = create_dict(who_world_case)
who_world_dict['case']['log'] = create_dict(who_world_log_case)
who_world_dict['death']['nolog'] = create_dict(who_world_death)
who_world_dict['death']['log'] = create_dict(who_world_log_death)
who_world_dict['case_per_100k_capita']['nolog'] = create_dict(who_world_case_per_100k_capita)
who_world_dict['case_per_100k_capita']['log'] = create_dict(who_world_log_case_per_100k_capita)
who_world_dict['death_per_100k_capita']['nolog'] = create_dict(who_world_death_per_100k_capita)
who_world_dict['death_per_100k_capita']['log'] = create_dict(who_world_log_death_per_100k_capita)

In [ ]:
who_world_dict

### 6. For Vulnerability Data

#### 6.0 Load Data

In [ ]:
vul_gdf = gpd.read_file('illinois/vulnerability.geojson')

#### 6.1 Get Constant Classes from today's data

In [ ]:
vul = vul_gdf['today_vul']

In [ ]:
vul_dict['case'] = {}

In [ ]:
vul_dict['case']['nolog'] = create_dict(vul)

In [ ]:
vul_dict

### 7. Summary and ouput

In [ ]:
import json
total_dict['county'] = county_dict
total_dict['state'] = state_dict
total_dict['dph_illinois'] = dph_illinois_dict
total_dict['who_world'] = who_world_dict
total_dict['vulnerability'] = vul_dict
# dynamic_dict['county'] = dynamic_county_dict
# dynamic_dict['state'] = dynamic_state_dict
# dynamic_dict['illinois'] = dynamic_illinois_dict
with open('classes.json','w') as json_file:
    json.dump(total_dict, json_file)
# with open('dynamic_classes.json','w') as json_file:
#     json.dump(dynamic_dict, json_file)
print('done')